# Cell 1: 노트북 개요

## 02_data_collection.ipynb - CARLA 데이터 수집

이 노트북에서는 CARLA 시뮬레이터에서 자율주행 학습용 데이터를 수집합니다.

### 수집 데이터
- **센서**: RGB 카메라 (전방)
- **상태**: 속도, 위치, 방향
- **규칙**: 신호등 상태, 속도 제한
- **액션 (정답)**: steer, throttle, brake (Expert Agent)
- **이벤트**: 충돌, 차선 침범

### 참고 논문
- **CARLA** (Dosovitskiy et al., CoRL 2017): 자율주행 연구용 시뮬레이터
- **End-to-End Learning** (Bojarski et al., 2016): 이미지→조향 직접 학습
- **DAgger** (Ross et al., 2011): Expert demonstration 수집 및 학습

In [ ]:
# Cell 1: 라이브러리 및 설정

import sys
import os
import time
import json
import logging
import numpy as np
import pandas as pd
import cv2
import yaml
from pathlib import Path
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# 프로젝트 루트
PROJECT_ROOT = Path().absolute().parent
sys.path.insert(0, str(PROJECT_ROOT))

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# CARLA 임포트
try:
    import carla
    print("✅ CARLA 패키지 로드 성공")
except ImportError:
    print("❌ CARLA 패키지를 설치해주세요: pip install carla==0.9.15")

# 설정 파일 로드
config_path = PROJECT_ROOT / 'config' / 'carla_config.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print(f"설정 로드 완료: {config_path}")
print(f"수집 설정: {config['collection']['num_episodes']}개 에피소드, 에피소드당 {config['collection']['episode_length']}프레임")

In [ ]:
# Cell 2: CARLA 클라이언트 연결

def connect_carla(host='localhost', port=2000, timeout=10.0):
    """CARLA 서버에 연결"""
    try:
        client = carla.Client(host, port)
        client.set_timeout(timeout)
        world = client.get_world()
        logger.info(f"CARLA 연결 성공: {world.get_map().name}")
        return client, world
    except Exception as e:
        logger.error(f"CARLA 연결 실패: {e}")
        return None, None

# 연결
client, world = connect_carla(
    host=config['server']['host'],
    port=config['server']['port'],
    timeout=config['server']['timeout']
)

if world:
    # 동기 모드 설정 (데이터 수집에 권장)
    settings = world.get_settings()
    settings.synchronous_mode = True
    settings.fixed_delta_seconds = 1.0 / config['collection']['fps']
    world.apply_settings(settings)
    print(f"✅ 동기 모드 활성화 (FPS: {config['collection']['fps']})")

In [ ]:
# Cell 3: 센서 설정 클래스

class SensorManager:
    """센서 관리 클래스"""
    
    def __init__(self, world, vehicle):
        self.world = world
        self.vehicle = vehicle
        self.sensors = {}
        self.data = {}
        self.blueprint_library = world.get_blueprint_library()
    
    def setup_camera(self, sensor_config):
        """RGB 카메라 설정"""
        camera_bp = self.blueprint_library.find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(sensor_config['width']))
        camera_bp.set_attribute('image_size_y', str(sensor_config['height']))
        camera_bp.set_attribute('fov', str(sensor_config['fov']))
        
        pos = sensor_config['position']
        rot = sensor_config.get('rotation', [0, 0, 0])
        transform = carla.Transform(
            carla.Location(x=pos[0], y=pos[1], z=pos[2]),
            carla.Rotation(pitch=rot[0], yaw=rot[1], roll=rot[2])
        )
        
        camera = self.world.spawn_actor(camera_bp, transform, attach_to=self.vehicle)
        camera.listen(lambda image: self._process_camera(image))
        
        self.sensors['rgb_front'] = camera
        logger.info(f"카메라 설정 완료: {sensor_config['width']}x{sensor_config['height']}")
    
    def setup_collision_sensor(self):
        """충돌 센서 설정"""
        collision_bp = self.blueprint_library.find('sensor.other.collision')
        collision_sensor = self.world.spawn_actor(
            collision_bp,
            carla.Transform(),
            attach_to=self.vehicle
        )
        collision_sensor.listen(lambda event: self._process_collision(event))
        self.sensors['collision'] = collision_sensor
        self.data['collision_history'] = []
        logger.info("충돌 센서 설정 완료")
    
    def setup_lane_invasion_sensor(self):
        """차선 침범 센서 설정"""
        lane_bp = self.blueprint_library.find('sensor.other.lane_invasion')
        lane_sensor = self.world.spawn_actor(
            lane_bp,
            carla.Transform(),
            attach_to=self.vehicle
        )
        lane_sensor.listen(lambda event: self._process_lane_invasion(event))
        self.sensors['lane_invasion'] = lane_sensor
        self.data['lane_invasion_history'] = []
        logger.info("차선 침범 센서 설정 완료")
    
    def _process_camera(self, image):
        """카메라 이미지 처리"""
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = array.reshape((image.height, image.width, 4))
        array = array[:, :, :3][:, :, ::-1]  # BGRA -> RGB
        self.data['current_image'] = array
        self.data['image_frame'] = image.frame
    
    def _process_collision(self, event):
        """충돌 이벤트 처리"""
        self.data['collision_history'].append({
            'frame': event.frame,
            'actor': event.other_actor.type_id if event.other_actor else 'unknown',
            'intensity': event.normal_impulse.length()
        })
    
    def _process_lane_invasion(self, event):
        """차선 침범 이벤트 처리"""
        self.data['lane_invasion_history'].append({
            'frame': event.frame,
            'lane_types': [str(x) for x in event.crossed_lane_markings]
        })
    
    def get_current_image(self):
        """현재 이미지 반환"""
        return self.data.get('current_image')
    
    def check_collision(self, frame):
        """해당 프레임에서 충돌 여부 확인"""
        for col in self.data.get('collision_history', []):
            if col['frame'] == frame:
                return True
        return False
    
    def check_lane_invasion(self, frame):
        """해당 프레임에서 차선 침범 여부 확인"""
        for lane in self.data.get('lane_invasion_history', []):
            if lane['frame'] == frame:
                return True
        return False
    
    def reset_events(self):
        """이벤트 기록 초기화"""
        self.data['collision_history'] = []
        self.data['lane_invasion_history'] = []
    
    def destroy(self):
        """모든 센서 삭제"""
        for name, sensor in self.sensors.items():
            if sensor is not None:
                sensor.stop()
                sensor.destroy()
        self.sensors = {}
        logger.info("센서 정리 완료")

print("✅ SensorManager 클래스 정의 완료")

In [ ]:
# Cell 4: 데이터 수집기 클래스

class DataCollector:
    """CARLA 데이터 수집기"""
    
    def __init__(self, world, save_dir):
        self.world = world
        self.save_dir = Path(save_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        
        self.vehicle = None
        self.sensor_manager = None
        self.current_episode = 0
        self.frames = []
        self.images = []
        
        # 블루프린트
        self.blueprint_library = world.get_blueprint_library()
        self.spawn_points = world.get_map().get_spawn_points()
    
    def spawn_vehicle(self, blueprint_name='vehicle.tesla.model3'):
        """차량 스폰"""
        vehicle_bp = self.blueprint_library.find(blueprint_name)
        spawn_point = np.random.choice(self.spawn_points)
        
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        logger.info(f"차량 스폰: {blueprint_name}")
        return self.vehicle
    
    def setup_sensors(self, sensor_config):
        """센서 설정"""
        self.sensor_manager = SensorManager(self.world, self.vehicle)
        self.sensor_manager.setup_camera(sensor_config)
        self.sensor_manager.setup_collision_sensor()
        self.sensor_manager.setup_lane_invasion_sensor()
    
    def get_vehicle_state(self):
        """차량 상태 정보 수집"""
        # 위치 및 방향
        transform = self.vehicle.get_transform()
        location = transform.location
        rotation = transform.rotation
        
        # 속도
        velocity = self.vehicle.get_velocity()
        speed = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
        
        # 컨트롤 (Expert action)
        control = self.vehicle.get_control()
        
        # 신호등
        traffic_light = 'none'
        if self.vehicle.is_at_traffic_light():
            tl = self.vehicle.get_traffic_light()
            state = tl.get_state()
            if state == carla.TrafficLightState.Red:
                traffic_light = 'red'
            elif state == carla.TrafficLightState.Yellow:
                traffic_light = 'yellow'
            elif state == carla.TrafficLightState.Green:
                traffic_light = 'green'
        
        # 속도 제한 (맵에서 가져오기 - 간소화)
        speed_limit = 50.0  # 기본값 km/h
        
        return {
            'ego_speed': speed,
            'ego_location': [location.x, location.y, location.z],
            'ego_rotation': [rotation.pitch, rotation.yaw, rotation.roll],
            'traffic_light': traffic_light,
            'speed_limit': speed_limit,
            'steer': control.steer,
            'throttle': control.throttle,
            'brake': control.brake
        }
    
    def collect_frame(self, frame_id):
        """단일 프레임 수집"""
        # 월드 틱 (동기 모드)
        world_frame = self.world.tick()
        
        # 약간 대기 (센서 데이터 수신)
        time.sleep(0.05)
        
        # 이미지
        image = self.sensor_manager.get_current_image()
        if image is None:
            return None
        
        # 상태
        state = self.get_vehicle_state()
        
        # 이벤트
        collision = self.sensor_manager.check_collision(world_frame)
        lane_invasion = self.sensor_manager.check_lane_invasion(world_frame)
        
        # 프레임 데이터
        frame_data = {
            'frame_id': frame_id,
            'world_frame': world_frame,
            'collision': collision,
            'lane_invasion': lane_invasion,
            **state
        }
        
        self.frames.append(frame_data)
        self.images.append(image.copy())
        
        return frame_data
    
    def run_episode(self, episode_length, enable_autopilot=True):
        """에피소드 실행"""
        self.frames = []
        self.images = []
        self.sensor_manager.reset_events()
        
        # Autopilot 활성화
        if enable_autopilot:
            self.vehicle.set_autopilot(True)
        
        # 안정화 대기
        for _ in range(10):
            self.world.tick()
        time.sleep(0.5)
        
        # 데이터 수집
        collision_count = 0
        for frame_id in tqdm(range(episode_length), desc=f"Episode {self.current_episode}"):
            frame_data = self.collect_frame(frame_id)
            
            if frame_data is None:
                continue
            
            # 충돌 시 카운트
            if frame_data['collision']:
                collision_count += 1
                if collision_count > 3:  # 3번 이상 충돌 시 에피소드 종료
                    logger.warning("다중 충돌로 에피소드 조기 종료")
                    break
        
        # Autopilot 비활성화
        self.vehicle.set_autopilot(False)
        
        return len(self.frames)
    
    def save_episode(self):
        """에피소드 저장"""
        episode_dir = self.save_dir / f"episode_{self.current_episode:03d}"
        episode_dir.mkdir(exist_ok=True)
        
        # 이미지 저장
        image_dir = episode_dir / 'images'
        image_dir.mkdir(exist_ok=True)
        
        for i, (frame, image) in enumerate(zip(self.frames, self.images)):
            image_path = image_dir / f"{i:06d}.jpg"
            cv2.imwrite(str(image_path), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            frame['image_path'] = str(image_path)
        
        # 메타데이터 저장
        df = pd.DataFrame(self.frames)
        df.to_parquet(episode_dir / 'frames.parquet', index=False)
        
        # 에피소드 정보
        episode_info = {
            'episode_id': self.current_episode,
            'num_frames': len(self.frames),
            'total_collisions': sum(1 for f in self.frames if f['collision']),
            'total_lane_invasions': sum(1 for f in self.frames if f['lane_invasion']),
            'timestamp': datetime.now().isoformat()
        }
        
        with open(episode_dir / 'metadata.json', 'w') as f:
            json.dump(episode_info, f, indent=2)
        
        logger.info(f"에피소드 {self.current_episode} 저장 완료: {len(self.frames)} 프레임")
        self.current_episode += 1
        
        return episode_info
    
    def reset(self):
        """에피소드 리셋 (차량 재스폰)"""
        # 기존 액터 삭제
        if self.sensor_manager:
            self.sensor_manager.destroy()
        if self.vehicle:
            self.vehicle.destroy()
        
        # 잠시 대기
        time.sleep(0.5)
    
    def cleanup(self):
        """완전 정리"""
        self.reset()
        logger.info("데이터 수집기 정리 완료")

print("✅ DataCollector 클래스 정의 완료")

In [ ]:
# Cell 5: 수집 실행 설정

# 저장 디렉토리
SAVE_DIR = PROJECT_ROOT / 'dataset' / 'carla_collected'
print(f"저장 디렉토리: {SAVE_DIR}")

# 수집 설정
NUM_EPISODES = config['collection']['num_episodes']  # 50
EPISODE_LENGTH = config['collection']['episode_length']  # 1000
SENSOR_CONFIG = config['sensors']['rgb_front']

print(f"\n수집 설정:")
print(f"  - 에피소드 수: {NUM_EPISODES}")
print(f"  - 에피소드당 프레임: {EPISODE_LENGTH}")
print(f"  - 예상 총 프레임: {NUM_EPISODES * EPISODE_LENGTH:,}")
print(f"  - 카메라 해상도: {SENSOR_CONFIG['width']}x{SENSOR_CONFIG['height']}")

In [ ]:
# Cell 6: 데이터 수집 실행

# 데이터 수집기 생성
collector = DataCollector(world, SAVE_DIR)

# 수집 통계
all_episodes_info = []

try:
    for ep in range(NUM_EPISODES):
        print(f"\n{'='*50}")
        print(f"에피소드 {ep + 1}/{NUM_EPISODES} 시작")
        print(f"{'='*50}")
        
        # 차량 스폰
        collector.spawn_vehicle()
        
        # 센서 설정
        collector.setup_sensors(SENSOR_CONFIG)
        
        # 에피소드 실행
        num_frames = collector.run_episode(EPISODE_LENGTH)
        
        # 저장
        episode_info = collector.save_episode()
        all_episodes_info.append(episode_info)
        
        # 리셋
        collector.reset()
        
        print(f"\n✅ 에피소드 {ep + 1} 완료:")
        print(f"   - 수집 프레임: {num_frames}")
        print(f"   - 충돌: {episode_info['total_collisions']}회")
        print(f"   - 차선 침범: {episode_info['total_lane_invasions']}회")

except KeyboardInterrupt:
    print("\n\n⚠️ 사용자에 의해 중단됨")

finally:
    collector.cleanup()
    
    # 동기 모드 해제
    settings = world.get_settings()
    settings.synchronous_mode = False
    world.apply_settings(settings)

print(f"\n\n{'='*50}")
print("데이터 수집 완료!")
print(f"총 {len(all_episodes_info)}개 에피소드 수집")

In [ ]:
# Cell 7: 수집 데이터 검증 및 시각화

import glob

# 수집된 에피소드 확인
episode_dirs = sorted(glob.glob(str(SAVE_DIR / 'episode_*')))
print(f"수집된 에피소드: {len(episode_dirs)}개\n")

# 통계 수집
total_frames = 0
total_collisions = 0
total_lane_invasions = 0

for ep_dir in episode_dirs:
    metadata_path = Path(ep_dir) / 'metadata.json'
    if metadata_path.exists():
        with open(metadata_path, 'r') as f:
            info = json.load(f)
        total_frames += info['num_frames']
        total_collisions += info['total_collisions']
        total_lane_invasions += info['total_lane_invasions']

print(f"총 프레임: {total_frames:,}")
print(f"총 충돌: {total_collisions}회")
print(f"총 차선 침범: {total_lane_invasions}회")
print(f"충돌률: {total_collisions / total_frames * 100:.2f}%" if total_frames > 0 else "")

In [ ]:
# Cell 8: 샘플 이미지 시각화

# 첫 번째 에피소드에서 샘플 이미지 로드
if episode_dirs:
    sample_ep = Path(episode_dirs[0])
    
    # 프레임 데이터 로드
    df = pd.read_parquet(sample_ep / 'frames.parquet')
    print(f"에피소드 데이터 shape: {df.shape}")
    print(f"\n컬럼: {list(df.columns)}")
    
    # 샘플 이미지 표시
    sample_indices = [0, len(df)//4, len(df)//2, 3*len(df)//4, len(df)-1]
    
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))
    
    for ax, idx in zip(axes, sample_indices):
        if idx < len(df):
            image_path = df.iloc[idx]['image_path']
            if os.path.exists(image_path):
                img = cv2.imread(image_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                ax.imshow(img)
                
                speed = df.iloc[idx]['ego_speed'] * 3.6  # m/s -> km/h
                steer = df.iloc[idx]['steer']
                ax.set_title(f"Frame {idx}\nSpeed: {speed:.1f} km/h\nSteer: {steer:.2f}")
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()
else:
    print("수집된 에피소드가 없습니다.")

In [ ]:
# Cell 9: 액션 분포 분석

if episode_dirs:
    # 모든 에피소드 데이터 합치기
    all_frames = []
    for ep_dir in episode_dirs[:5]:  # 처음 5개만
        df = pd.read_parquet(Path(ep_dir) / 'frames.parquet')
        all_frames.append(df)
    
    combined_df = pd.concat(all_frames, ignore_index=True)
    
    # 액션 분포
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Steer
    axes[0].hist(combined_df['steer'], bins=50, alpha=0.7, edgecolor='black')
    axes[0].set_xlabel('Steer [-1, 1]')
    axes[0].set_ylabel('Count')
    axes[0].set_title(f'Steer Distribution\nMean: {combined_df["steer"].mean():.3f}')
    axes[0].axvline(0, color='red', linestyle='--', alpha=0.5)
    
    # Throttle
    axes[1].hist(combined_df['throttle'], bins=50, alpha=0.7, edgecolor='black', color='green')
    axes[1].set_xlabel('Throttle [0, 1]')
    axes[1].set_ylabel('Count')
    axes[1].set_title(f'Throttle Distribution\nMean: {combined_df["throttle"].mean():.3f}')
    
    # Brake
    axes[2].hist(combined_df['brake'], bins=50, alpha=0.7, edgecolor='black', color='red')
    axes[2].set_xlabel('Brake [0, 1]')
    axes[2].set_ylabel('Count')
    axes[2].set_title(f'Brake Distribution\nMean: {combined_df["brake"].mean():.3f}')
    
    plt.tight_layout()
    plt.show()
    
    # 속도 분포
    plt.figure(figsize=(10, 4))
    plt.hist(combined_df['ego_speed'] * 3.6, bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel('Speed (km/h)')
    plt.ylabel('Count')
    plt.title(f'Speed Distribution\nMean: {combined_df["ego_speed"].mean() * 3.6:.1f} km/h')
    plt.axvline(50, color='red', linestyle='--', label='Speed Limit (50 km/h)')
    plt.legend()
    plt.show()
    
    # 신호등 상태 분포
    plt.figure(figsize=(8, 4))
    tl_counts = combined_df['traffic_light'].value_counts()
    colors = {'none': 'gray', 'green': 'green', 'yellow': 'yellow', 'red': 'red'}
    tl_counts.plot(kind='bar', color=[colors.get(x, 'blue') for x in tl_counts.index])
    plt.xlabel('Traffic Light State')
    plt.ylabel('Count')
    plt.title('Traffic Light Distribution')
    plt.xticks(rotation=0)
    plt.show()

In [ ]:
# Cell 10: 데이터 저장 요약

# 전체 데이터셋 정보 저장
dataset_info = {
    'num_episodes': len(episode_dirs),
    'total_frames': total_frames,
    'total_collisions': total_collisions,
    'total_lane_invasions': total_lane_invasions,
    'sensor_config': SENSOR_CONFIG,
    'collection_date': datetime.now().isoformat(),
    'carla_version': '0.9.15'
}

with open(SAVE_DIR / 'dataset_info.json', 'w') as f:
    json.dump(dataset_info, f, indent=2)

print("=" * 50)
print("데이터 수집 완료 요약")
print("=" * 50)
print(f"저장 위치: {SAVE_DIR}")
print(f"에피소드 수: {len(episode_dirs)}")
print(f"총 프레임: {total_frames:,}")
print(f"\n다음 단계: 03_kitti_exploration.ipynb")

## 수집 데이터 구조

```
dataset/carla_collected/
├── dataset_info.json          # 전체 데이터셋 정보
├── episode_000/
│   ├── metadata.json          # 에피소드 메타데이터
│   ├── frames.parquet         # 프레임별 데이터
│   └── images/
│       ├── 000000.jpg
│       ├── 000001.jpg
│       └── ...
├── episode_001/
│   └── ...
└── ...
```

## 참고 논문

| 논문 | 핵심 아이디어 |
|------|---------------|
| CARLA (Dosovitskiy et al., 2017) | 자율주행 시뮬레이터, 다양한 센서 및 환경 조건 제공 |
| End-to-End Learning (Bojarski et al., 2016) | CNN으로 이미지→조향 직접 학습, NVIDIA PilotNet |
| DAgger (Ross et al., 2011) | Dataset Aggregation, 분포 불일치 해결 |